# 初期設定

* ヒアドキュメントで吐き出したhtmlをscpで直接アクセスできるサーバに転送するための設定
    * セキュリティー的にあまりよくないので、内部のみアクセスできるサーバに漏れても良い検証用サーバのパスワード利用を推奨

# Jupyterで出力したファイルを転送して直接ブラウザで確認するための設定


## sshpassのインストール

```
sudo apt-get install sshpass
```

## ファイルを転送

```
### sshpass -p "(サーバのパスワード)" scp (転送するhtmlファイル) (サーバのユーザ)@(サーバのアドレス):(転送先)
例
sshpass -p "pass" scp jquery.html ftakao2007@192.168.33.11:/home/ftakao2007
```

In [11]:
%%bash

### 各種認証情報が入ったファイルを作成
cat <<- EOS > ${HOME}/secrets
pass="pass"
user="ftakao2007"
host="192.168.33.11"
path="/home/ftakao2007"
EOS

### ファイルの中身を表示
cat ${HOME}/secrets

### 認証情報を変数に格納
source ${HOME}/secrets

### scpに利用
echo "sshpass -p \"${pass}\" scp jquery.html ${user}@${host}:${path}"

pass="pass"
user="ftakao2007"
host="192.168.33.11"
path="/home/ftakao2007"
sshpass -p "pass" scp jquery.html ftakao2007@192.168.33.11:/home/ftakao2007


# nodeを入れてコマンドでjQueryを実行するための設定

いちいち転送してブラウザ起動して確認しなくても良くなる

## インストール

* 安全のためコメントアウトしてます

In [11]:
%%bash
### Ubuntu16.04, node v8.1.1, jquery@3.2.1, jsdom@11.0.0

### nvmのインストール
#git clone https://github.com/creationix/nvm.git ~/.nvm
#source ~/.nvm/nvm.sh

### 最新のnodeをインストール
#nvm install `nvm ls-remote | tail -1`
#node -v

### package.jsonの作成
### npm initに相当
# cat <<- 'EOS' > package.json
# {
#   "name": "jquery_sample",
#   "version": "1.0.0",
#   "description": "",
#   "main": "index.js",
#   "scripts": {
#     "test": "echo \"Error: no test specified\" && exit 1"
#   },
#   "author": "",
#   "license": "ISC"
# }
# EOS

### jqueryとjsdomのインストール
#npm install jquery jsdom --save

+ jquery@3.2.1
+ jsdom@11.0.0
added 93 packages in 11.584s


npm notice created a lockfile as package-lock.json. You should commit this file.
npm WARN jquery_sample@1.0.0 No description
npm WARN jquery_sample@1.0.0 No repository field.



In [12]:
%%bash
source ~/.nvm/nvm.sh

### サンプルファイルの作成
cat <<- 'EOS' > fruits.js
const jsdom = require("jsdom");
const { JSDOM } = jsdom;
const { window } = new JSDOM(`<!DOCTYPE html>`);
const $ = require('jquery')(window);

$(function() {

    var arr = [ "apple", "orange", "grape" ];
    $.each(arr, function(i, val) {
        console.log(i+':'+val);
    });
});
EOS

### 実行
node fruits.js

0:apple
1:orange
2:grape


## htmlを読み込んで値を取り出す

* index.htmlを読み込んでbodyの内容とjQueryでh1タグ中のテキストを取得して出力する

In [18]:
%%bash
source ~/.nvm/nvm.sh

### htmlファイルの作成
cat <<- 'EOS' > index.html
<html>
  <body>
    <div id="hoge">
      <h1>h1 text</h1>
    </div>
  </body>
</html>
EOS

### jsファイルの作成
cat <<- 'EOS' > app.js
const jsdom = require("jsdom");
const { JSDOM } = jsdom;
const { window } = new JSDOM(`<!DOCTYPE html>`);
const $ = require('jquery')(window);
const fs = require('fs');

fs.readFile(__dirname + '/index.html', function (err, data) {
  if (err) throw err;

  var body = data.toString();
  console.log(body);
  console.log($(body).find('h1').text());
});
EOS

### 実行
node app.js

<html>
  <body>
    <div id="hoge">
      <h1>h1 text</h1>
    </div>
  </body>
</html>

h1 text
